In [1]:
#Import dependencies
import pandas as pd
import sqlite3 as sqlite
from datetime import datetime as dt
import numpy as np

#Path name files to datasets for cleaning and processing.  Jeopardy CSV is from Kaggle, Contestants and Locations
# are from github/anuparna/jeopardy.  jeaopardy_archive is the flat file Geetha found
path = ('./Resources/jeopardy_archive_data.csv')
path2 = ('./Resources/JEOPARDY_CSV.csv')
path3 = ('./Resources/contestants.csv')
path4 = ('./Resources/locations.csv')
path5 = ('./Resources/final_results.csv')
path6 = ('./Resources/cleaned_data.csv')

In [2]:
#Connector to sqlite3.  Creates Jeopardy database
conn = sqlite.connect('jeopardy_db.sqlite')

In [3]:
from numpy import datetime64

#Read and format csv files into dataframe.  
#Renamed the columns in 'JEOPARDY_CSV.csv' to find fields to join the two datasets.

df = pd.read_csv(path)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)
df4 = pd.read_csv(path4)
df5 = pd.read_csv(path5)
df6 = pd.read_csv(path6)

jeopardy_df1 = pd.DataFrame(df)
jeopardy_df1['date'] = pd.to_datetime(jeopardy_df1['date'], errors='coerce')
jeopardy_df1['date'] = jeopardy_df1['date'].dt.strftime('%m/%d/%Y')
jeopardy_df1 = pd.DataFrame(jeopardy_df1).rename(columns={'full_name': 'Name'})

#Removed extra text from jeopardy_archive data to join fields on "show_info"

jeopardy_df1['show_info'] = jeopardy_df1['show_info'].str.replace(r'\D','')
jeopardy_df1['show_info'] = jeopardy_df1['show_info'].astype('int')
num=100000000
jeopardy_df1['show_info'] =(jeopardy_df1['show_info']/num).round(0)
jeopardy_df1['show_info'] = jeopardy_df1['show_info'].astype('int')


jeopardy_df2 = pd.DataFrame(df2).rename(columns={'Show Number': 'show_info', ' Air Date': 'date', ' Round': 'Round_', ' Category': 'Category', ' Value': 'Value',
' Question': 'Question', ' Answer': 'Answer'})
jeopardy_df2['date'] = pd.to_datetime(jeopardy_df2['date'], errors='coerce')
jeopardy_df2['date'] = jeopardy_df2['date'].dt.strftime('%m/%d/%Y')

jeopardy_df3 = pd.DataFrame(df3)
jeopardy_df3['Name'] = jeopardy_df3['player_first_name'].map(str) + ' ' + jeopardy_df3['player_last_name'].map(str)
jeopardy_df3 = jeopardy_df3.drop(columns = ['player_first_name', 'player_last_name'])
jeopardy_df3 = jeopardy_df3.drop_duplicates(subset = 'Name')

jeopardy_df4 = pd.DataFrame(df4)
jeopardy_df5 = pd.DataFrame(df5)

jeopardy_df6 = pd.DataFrame(df6)

/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.


In [4]:
#Read datasets into Sqlite table

jeopardy_df1.to_sql('jeopardy_archvie_tbl', conn, if_exists='replace', index = False)
jeopardy_df2.to_sql('jeopardy_kaggle_tbl', conn, if_exists='replace', index = False)
jeopardy_df3.to_sql('jeopardy_contestants_tbl', conn, if_exists='replace', index = False)
jeopardy_df4.to_sql('jeopardy_seat_locations_tbl', conn, if_exists='replace', index = False)
jeopardy_df5.to_sql('jeopardy_final_results_tbl', conn, if_exists='replace', index = False)
jeopardy_df6.to_sql('jeopardy_cleaned_data_tbl', conn, if_exists='replace', index = False)
conn.commit()


In [5]:
#Check headers to use in SQL queries
print(f'jeopardy_archive:', list(jeopardy_df1.columns))
print(f' JEOPARDY_CSV:', list(jeopardy_df2.columns))
print(f' contestants:',list(jeopardy_df3.columns))
print(f' locations', list(jeopardy_df4.columns))
print(f' final_results', list(jeopardy_df5.columns))
print(f'cleaned_data', list(jeopardy_df6.columns))

jeopardy_archive: ['nickname', 'player_details', 'final_score', 'show_info', 'Name', 'occupation', 'hometown', 'archive_info', 'date']
 JEOPARDY_CSV: ['show_info', 'date', 'Round_', 'Category', 'Value', 'Question', 'Answer']
 contestants: ['player_id', 'hometown_city', 'hometown_state', 'occupation', 'Name']
 locations ['game_id', 'player_id', 'seat_location', 'season']
 final_results ['game_id', 'season', 'position', 'dj_score', 'wager', 'correct', 'coryat_score']
cleaned_data ['full_name', 'final_score', 'occupation', 'city', 'state', 'state_id', 'archive_info', 'date', 'dt_indx', 'winner_flg', 'gender']


In [6]:
jeopardy_df1.head(5)

,nickname,player_details,final_score,show_info,Name,occupation,hometown,archive_info,date
0,Jason,"Jason Zuffranieri, a math teacher from Albuque...","$27,600",8045,Jason Zuffranieri,a math teacher,"Albuquerque, New Mexico",J! Archive - Show #8045,07/26/2019
1,Maggie,"Maggie Lehrman, an editor and writer from Broo...",$0,8045,Maggie Lehrman,an editor and writer,"Brooklyn, New York",J! Archive - Show #8045,07/26/2019
2,Michael,"Michael Riggs, an educational therapist from T...",$2,8045,Michael Riggs,an educational therapist,"Tustin, California",J! Archive - Show #8045,07/26/2019
3,Jason,"Jason Zuffranieri, a math teacher from Albuque...","$4,400",8044,Jason Zuffranieri,a math teacher,"Albuquerque, New Mexico",J! Archive - Show #8044,07/25/2019
4,Eric,"Eric Kaplan, a retired OB/GYN physician from L...",$0,8044,Eric Kaplan,a retired OB/GYN physician,"Long Beach, California",J! Archive - Show #8044,07/25/2019


In [7]:
jeopardy_df2.head(5)

,show_info,date,Round_,Category,Value,Question,Answer
0,4680,12/31/2004,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,12/31/2004,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,12/31/2004,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,12/31/2004,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,12/31/2004,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [8]:
jeopardy_df3.head(5)

,player_id,hometown_city,hometown_state,occupation,Name
0,11367,Los Angeles,California,a writer,Seth Madej
1,11368,Norwalk,Connecticut,a manager of commercial support content,Rich Steeves
2,11363,New York,New York,a financial analyst,Ragavan Ramsubramani
3,11365,San Jose,California,a library media center technician,Laura Spoelstra
4,11366,Baltimore,Maryland,a professor,Erin Wilson


jeopardy_df4.head(5)

In [9]:
jeopardy_df6.head(5)

,full_name,final_score,occupation,city,state,state_id,archive_info,date,dt_indx,winner_flg,gender
0,Jason Zuffranieri,27600,a math teacher,Albuquerque,New Mexico,NM,J! Archive - Show #8045,2019-07-26,1,1,male
1,Jason Zuffranieri,4400,a math teacher,Albuquerque,New Mexico,NM,J! Archive - Show #8044,2019-07-25,1,1,male
2,Jason Zuffranieri,30000,a math teacher,Albuquerque,New Mexico,NM,J! Archive - Show #8043,2019-07-24,1,1,male
3,Jason Zuffranieri,30100,a math teacher,Albuquerque,New Mexico,NM,J! Archive - Show #8042,2019-07-23,1,1,male
4,Jason Zuffranieri,18600,a math teacher,Albuquerque,New Mexico,NM,J! Archive - Show #8041,2019-07-22,1,1,male


In [10]:
SQL_join1 = pd.read_sql(' \
SELECT B.Name, A.seat_location, C.correct, D.show_info  \
FROM jeopardy_seat_locations_tbl A \
LEFT JOIN jeopardy_contestants_tbl B ON A.player_id = B.player_id \
INNER JOIN jeopardy_final_results_tbl C ON C.game_id = A.game_id \
INNER JOIN jeopardy_archvie_tbl D ON B.Name = D.Name'
                        
                        
                        
                        
                        , conn)  

SQL_join1.to_sql('jeopardy_sql_query1', conn, if_exists='replace', index = False)
conn.commit()

In [11]:
SQL_join2 = pd.read_sql(' \
SELECT Q.Name, Q.seat_location, Q.correct, Q.show_info, A. Category, A.question, A.Answer, A.value  \
FROM jeopardy_sql_query1 Q \
LEFT JOIN jeopardy_kaggle_tbl A ON Q.show_info = A.show_info', conn)

SQL_join2.dropna().sort_values(by= 'show_info').to_sql('jeopardy_sql_query2', conn, if_exists='replace', index = False)
conn.commit()



# Preprocessing...

In [12]:
winner_by_occupation = pd.read_sql('SELECT final_score, occupation, city, winner_flg FROM jeopardy_cleaned_data_tbl', conn)
# SQL_winner_by_occupation = pd.read_sql('\
# SELECT occupation,count(occupation) as total FROM jeopardy_cleaned_data_tbl \
# WHERE winner_flg = 1 \
# GROUP BY occupation \
# ORDER BY total DESC', conn)

In [13]:
winner_by_occupation

,final_score,occupation,city,winner_flg
0,27600,a math teacher,Albuquerque,1
1,4400,a math teacher,Albuquerque,1
2,30000,a math teacher,Albuquerque,1
3,30100,a math teacher,Albuquerque,1
4,18600,a math teacher,Albuquerque,1
...,...,...,...,...
3142,2700,a middle school librarian,Summerville,0
3143,5999,a jeweler,Caribou,0
3144,11401,a controller,Troy,0
3145,3598,a high school language arts teacher,Chaparral,0


In [14]:
winner_by_occupation.shape

(3147, 4)

In [15]:
winner_by_occupation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3147 entries, 0 to 3146
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   final_score  3147 non-null   int64 
 1   occupation   3147 non-null   object
 2   city         3147 non-null   object
 3   winner_flg   3147 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 98.5+ KB


In [16]:
winner_by_occupation.columns

Index(['final_score', 'occupation', 'city', 'winner_flg'], dtype='object')

In [17]:
winner_by_occupation

,final_score,occupation,city,winner_flg
0,27600,a math teacher,Albuquerque,1
1,4400,a math teacher,Albuquerque,1
2,30000,a math teacher,Albuquerque,1
3,30100,a math teacher,Albuquerque,1
4,18600,a math teacher,Albuquerque,1
...,...,...,...,...
3142,2700,a middle school librarian,Summerville,0
3143,5999,a jeweler,Caribou,0
3144,11401,a controller,Troy,0
3145,3598,a high school language arts teacher,Chaparral,0


In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_app_df = pd.get_dummies(winner_by_occupation)
numeric_app_df.columns = numeric_app_df.columns.str.replace(' ', '_')
numeric_app_df

,final_score,winner_flg,occupation_a_10th_and_11th_grade_English_teacher,occupation_a_10th_grade_English_teacher,occupation_a_10th_grade_chemistry_teacher,occupation_a_12-year-old,occupation_a_12th_grade_English_language_arts_teacher,occupation_a_1st_grade_teacher,occupation_a_20-time_Jeopardy!_champion,occupation_a_4th_grade_language_arts_teacher,...,city_Woodbridge,city_Woodbury,city_Woodinville,city_Woodside,city_Worcester,city_Worthington,city_Wyandotte,city_Wyncote,city_York,city_Ypsilanti
0,27600,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4400,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30100,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18600,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,2700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3143,5999,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3144,11401,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3145,3598,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
print(numeric_app_df.columns)


Index(['final_score', 'winner_flg',
       'occupation_a_10th_and_11th_grade_English_teacher',
       'occupation_a_10th_grade_English_teacher',
       'occupation_a_10th_grade_chemistry_teacher', 'occupation_a_12-year-old',
       'occupation_a_12th_grade_English_language_arts_teacher',
       'occupation_a_1st_grade_teacher',
       'occupation_a_20-time_Jeopardy!_champion',
       'occupation_a_4th_grade_language_arts_teacher',
       ...
       'city_Woodbridge', 'city_Woodbury', 'city_Woodinville', 'city_Woodside',
       'city_Worcester', 'city_Worthington', 'city_Wyandotte', 'city_Wyncote',
       'city_York', 'city_Ypsilanti'],
      dtype='object', length=2131)


In [20]:
from sklearn.model_selection import train_test_split

# Split our preprocessed data into our features and target arrays
y = numeric_app_df["winner_flg"].values
x = numeric_app_df.drop(["winner_flg"],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=100, stratify=y)

In [21]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
import tensorflow as tf

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 30
hidden_nodes_layer2 =45

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

2023-06-05 21:39:08.179132: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                63930     
                                                                 
 dense_1 (Dense)             (None, 45)                1395      
                                                                 
 dense_2 (Dense)             (None, 1)                 46        
                                                                 
Total params: 65,371
Trainable params: 65,371
Non-trainable params: 0
_________________________________________________________________


2023-06-05 21:39:11.396618: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=500)

Epoch 1/500
74/74 [==============================] - 1s 2ms/step - loss: 0.6795 - accuracy: 0.5568
Epoch 2/500
74/74 [==============================] - 0s 1ms/step - loss: 0.5144 - accuracy: 0.7258
Epoch 3/500
74/74 [==============================] - 0s 1ms/step - loss: 0.3971 - accuracy: 0.8131
Epoch 4/500
74/74 [==============================] - 0s 1ms/step - loss: 0.3154 - accuracy: 0.8483
Epoch 5/500
74/74 [==============================] - 0s 1ms/step - loss: 0.2701 - accuracy: 0.8712
Epoch 6/500
74/74 [==============================] - 0s 1ms/step - loss: 0.2387 - accuracy: 0.8826
Epoch 7/500
74/74 [==============================] - 0s 1ms/step - loss: 0.2182 - accuracy: 0.8928
Epoch 8/500
74/74 [==============================] - 0s 1ms/step - loss: 0.1939 - accuracy: 0.9064
Epoch 9/500
74/74 [==============================] - 0s 1ms/step - loss: 0.1865 - accuracy: 0.9106
Epoch 10/500
74/74 [==============================] - 0s 1ms/step - loss: 0.1716 - accuracy: 0.9131
Epoch 11/

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

25/25 - 0s - loss: 4.8298 - accuracy: 0.7078 - 159ms/epoch - 6ms/step
Loss: 4.829838275909424, Accuracy: 0.7077509760856628


In [26]:
#Predicting
y_pred = nn.predict(X_test_scaled)
print(y_pred)

25/25 [==============================] - 0s 840us/step
[[1.00000000e+00]
 [9.99608517e-01]
 [1.11531382e-02]
 [3.26084841e-12]
 [3.94138680e-11]
 [9.38785938e-09]
 [1.26110178e-10]
 [1.14579161e-04]
 [2.74136473e-05]
 [6.42711461e-01]
 [1.12314444e-12]
 [2.93959033e-22]
 [3.98318768e-02]
 [5.77899875e-15]
 [5.65786421e-01]
 [4.23626840e-01]
 [2.02289463e-10]
 [2.52801001e-01]
 [9.99999583e-01]
 [1.67533235e-20]
 [7.15963733e-09]
 [1.25015858e-15]
 [9.99998927e-01]
 [1.00000000e+00]
 [9.26681221e-01]
 [9.99985814e-01]
 [1.42201399e-14]
 [9.99886632e-01]
 [8.04339722e-03]
 [1.84722827e-03]
 [1.34116504e-27]
 [1.00000000e+00]
 [9.99769509e-01]
 [3.30323128e-05]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [3.93832516e-20]
 [6.28848553e-01]
 [1.00000000e+00]
 [0.00000000e+00]
 [2.91493595e-01]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.37972394e-02]
 [9.99990344e-01]
 [1.66083884e-03]
 [1.30224496e-01]
 [2.69720485e-34]
 [8.71217787e-01]
 [4.36488783e-08]
 [4.44848510e-18]
 [5.13766

In [27]:
print("number of test samples :", len(X_test_scaled))
print("number of training samples:", len(X_train_scaled))

number of test samples : 787
number of training samples: 2360


In [28]:
X_test_scaled.shape

(787, 2130)

In [29]:
X_train_scaled.shape

(2360, 2130)

In [30]:
# set random seed for reproducibility 
# see: https://stackoverflow.com/questions/21494489/what-does-numpy-random-seed0-do
np.random.seed(45)

In [31]:
y_train.shape

(2360,)

In [32]:
y_train[0]


0

In [33]:
# Export our model to HDF5 file
nn.save('Models/winnerbyoccupation.h5')


# Generating predictions



# I have failed in apply my model in a way that I could answer my question. 
